In [1]:
!python --version

Python 3.10.11


In [2]:
import langchain
langchain.__version__

'0.3.14'

In [3]:
from langchain.agents import initialize_agent, Tool
from langchain.llms import OpenAI

from dotenv import load_dotenv
import os

load_dotenv()
openai_api_key = os.getenv("OPENAI_API_KEY")

In [4]:
#!pip install -U duckduckgo-search

from langchain_community.tools.ddg_search.tool import DuckDuckGoSearchRun

# Web Search Tool using DuckDuckGo
search_tool = Tool(
    name="DuckDuckGo Search",
    func=DuckDuckGoSearchRun().run,
    description="Use this tool to perform web searches using DuckDuckGo."
)



llm = OpenAI(temperature=0.5)

tools = [search_tool]

agent = initialize_agent(tools, llm, agent="zero-shot-react-description", verbose=True)

response = agent.run("What are the top trending movies this week?")

print(response)

C:\Users\f\AppData\Local\Temp\ipykernel_16492\807594551.py:14: LangChainDeprecationWarning: The class `OpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAI``.
  llm = OpenAI(temperature=0.5)
C:\Users\f\AppData\Local\Temp\ipykernel_16492\807594551.py:18: LangChainDeprecationWarning: LangChain agents will continue to be supported, but it is recommended for new use cases to be built with LangGraph. LangGraph offers a more flexible and full-featured framework for building agents, including support for tool-calling, persistence of state, and human-in-the-loop workflows. For details, refer to the `LangGraph documentation <https://langchain-ai.github.io/langgraph/>`_ as well as guides for `Migrating from AgentExecutor <https://python.langchain.com/docs/h



> Entering new AgentExecutor chain...
 I should use a search engine to find this information
Action: DuckDuckGo Search
Action Input: "top trending movies this week"
Observation: Prime Video's current No. 1 movie is an uplifting sports movie called "Unstoppable", and Netflix is hoping to challenge its streaming rival with a similarly inspirational sports drama of its ... The best new movies to stream this week include A24's A Different Man on Max, Back in Action on Netflix, A Real Pain on Hulu, and more. The best new movies to watch on streaming this week feature some big-name stars as well as some of the most acclaimed performances of 2024. Movies from all types of genres are available on streamers like Netflix, Amazon Prime Video, Max, Peacock, and Paramount+, with titles that will appeal to audiences of all ages and all tastes. ... TOP TRENDING This week's most consumed, searched and shared content SOURCE: ... Top Movies The most-viewed films of the week. 1 Carry-On. Watch now: Net

In [7]:
# The WikipediaAPIWrapper - allows you to query Wikipedia for articles and summaries.

#!pip install wikipedia

from langchain_community.utilities.wikipedia import WikipediaAPIWrapper

# Step 1: Initialize the Wikipedia Tool
wikipedia_tool = Tool(
    name="Wikipedia Search",
    func=WikipediaAPIWrapper().run,
    description="Use this tool to search Wikipedia for information on any topic."
)

llm = OpenAI(temperature=0.5)

tools = [wikipedia_tool]

agent = initialize_agent(
    tools=tools,
    llm=llm,
    agent="zero-shot-react-description",
    verbose=True
)

query = "Who is Alan Turing, and what are his contributions?"
response = agent.run(query)
print("Response from Agent:")
print(response)



> Entering new AgentExecutor chain...
 I should always think about what to do
Action: Wikipedia Search
Action Input: Alan Turing
Observation: Page: Alan Turing
Summary: Alan Mathison Turing (; 23 June 1912 – 7 June 1954) was an English mathematician, computer scientist, logician, cryptanalyst, philosopher and theoretical biologist. He was highly influential in the development of theoretical computer science, providing a formalisation of the concepts of algorithm and computation with the Turing machine, which can be considered a model of a general-purpose computer. Turing is widely considered to be the father of theoretical computer science.
Born in London, Turing was raised in southern England. He graduated from King's College, Cambridge, and in 1938, earned a doctorate degree from Princeton University. During World War II, Turing worked for the Government Code and Cypher School at Bletchley Park, Britain's codebreaking centre that produced Ultra intelligence. He led Hut 8, the secti

In [8]:
import sqlite3

# Connect to SQLite database (creates example.db if it doesn't exist)
connection = sqlite3.connect("example.db")
cursor = connection.cursor()

# Create the inventory table
cursor.execute("""
CREATE TABLE IF NOT EXISTS inventory (
    id INTEGER PRIMARY KEY,
    product_name TEXT NOT NULL,
    stock_quantity INTEGER NOT NULL
)
""")

# Insert sample data
cursor.executemany("""
INSERT INTO inventory (product_name, stock_quantity) VALUES (?, ?)
""", [
    ("Product A", 150),
    ("Product B", 200),
    ("Product C", 300),
    ("Product D", 100),
    ("Product E", 50)
])

# Commit and close connection
connection.commit()
connection.close()

print("Database created and populated with sample data.")


Database created and populated with sample data.


In [9]:
from langchain_community.utilities.sql_database import SQLDatabase
from langchain_community.agent_toolkits.sql.toolkit import SQLDatabaseToolkit

In [10]:
# Step 1: Connect to the Database
db = SQLDatabase.from_uri("sqlite:///example.db")

# Step 3: Initialize the LLM
llm = OpenAI(temperature=0.5)

# Step 2: Create the SQL Database Toolkit

toolkit = SQLDatabaseToolkit(db=db, llm=llm)

tools = toolkit.get_tools()

agent = initialize_agent(
    tools=tools,
    llm=llm,
    agent="zero-shot-react-description",
    verbose=True
)


query1 = "How many units of Product A are in stock?"
response1 = agent.run(query1)
print("Response:", response1)



> Entering new AgentExecutor chain...
 I need to retrieve the number of units of Product A from the database.
Action: sql_db_query
Action Input: SELECT units FROM products WHERE product_name = 'Product A'
Observation: Error: (sqlite3.OperationalError) no such table: products
[SQL: SELECT units FROM products WHERE product_name = 'Product A']
(Background on this error at: https://sqlalche.me/e/20/e3q8)
 I need to use sql_db_list_tables to check if the 'products' table exists.
Action: sql_db_list_tables
Action Input: 
Observation: inventory
 I need to use sql_db_schema to get the schema and sample rows for the 'products' table.
Action: sql_db_schema
Action Input: products
Observation: Error: table_names {'products'} not found in database
 I need to double check my query using sql_db_query_checker.
Action: sql_db_query_checker
Action Input: SELECT units FROM inventory WHERE product_name = 'Product A'
Observation: 

SELECT units FROM inventory WHERE product_name = 'Product A'
 My query is

In [5]:
# ArxivQueryRun
from langchain.agents import initialize_agent, Tool
from langchain.llms import OpenAI

from dotenv import load_dotenv
import os

load_dotenv()
openai_api_key = os.getenv("OPENAI_API_KEY")


from langchain_community.tools.arxiv.tool import ArxivQueryRun

arxiv_tool = Tool(
    name="Arxiv Search",
    func=ArxivQueryRun().run,
    description="Use this tool to search academic papers on arXiv. Provide topics, authors, or keywords."
)

# Step 2: Initialize the LLM
llm = OpenAI(temperature=0.5)

# Step 3: Combine the tool into a list
tools = [arxiv_tool]

# Step 4: Initialize the Agent
agent = initialize_agent(
    tools=tools,
    llm=llm,
    agent="zero-shot-react-description",
    verbose=True
)

# Step 5: Query the Agent
query = "Search for research papers on Diffusion Models in 2024."
response = agent.run(query)
print("Response from Agent:")
print(response)




> Entering new AgentExecutor chain...
 I should use the Arxiv Search tool to search for academic papers.
Action: Arxiv Search
Action Input: topics = 'diffusion models', start_year = 2024
Observation: Published: 2024-03-11
Title: TopicDiff: A Topic-enriched Diffusion Approach for Multimodal Conversational Emotion Detection
Authors: Jiamin Luo, Jingjing Wang, Guodong Zhou
Summary: Multimodal Conversational Emotion (MCE) detection, generally spanning across
the acoustic, vision and language modalities, has attracted increasing interest
in the multimedia community. Previous studies predominantly focus on learning
contextual information in conversations with only a few considering the topic
information in single language modality, while always neglecting the acoustic
and vision topic information. On this basis, we propose a model-agnostic
Topic-enriched Diffusion (TopicDiff) approach for capturing multimodal topic
information in MCE tasks. Particularly, we integrate the diffusion model in

In [6]:
from langchain_community.tools.file_management import ReadFileTool
from langchain.agents import initialize_agent, Tool
from langchain.llms import OpenAI

from dotenv import load_dotenv
import os

load_dotenv()
openai_api_key = os.getenv("OPENAI_API_KEY")

# Step 1: Initialize the ReadFileTool
read_file_tool = Tool(
    name="Read File",
    func=ReadFileTool().run,
    description="Use this tool to read the content of a text file in the current working directory. Provide the file name as input."
)

# Step 2: Initialize the LLM
llm = OpenAI(temperature=0.5)

# Step 3: Combine the tool into a list
tools = [read_file_tool]

# Step 4: Initialize the Agent
agent = initialize_agent(
    tools=tools,
    llm=llm,
    agent="zero-shot-react-description",
    verbose=True
)

# Step 5: Query the Agent with a file name
file_name = "example.txt"

response = agent.run(f"Read the content of the file named {file_name}.")
print("Response from Agent:")
print(response)




> Entering new AgentExecutor chain...
 You should always think about the file name and location.
Action: Read File
Action Input: example.txt
Observation: This is a sample data
 Now I know the content of the file.
Final Answer: This is a sample data

> Finished chain.
Response from Agent:
This is a sample data
